<a href="https://colab.research.google.com/github/csce585-mlsystems/CSCE585ProjectROI/blob/michelleBranch/Current_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> Running experiments in ModelDevelopment Directory



Cloned the repository

In [ ]:
!git clone https://github.com/csce585-mlsystems/CSCE585ProjectROI

Cloning into 'CSCE585ProjectROI'...
remote: Enumerating objects: 678, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 678 (delta 61), reused 44 (delta 44), pack-reused 611 (from 1)
Receiving objects: 100% (678/678), 1.54 MiB | 18.16 MiB/s, done.
Resolving deltas: 100% (325/325), done.


After cloning, you might need to navigate into the repository's directory. Then, install any necessary Python packages. Often, these are listed in a `requirements.txt` file. You may need to check the repository's documentation for the exact file name or commands.

In [ ]:
# Repo directories
import os
os.chdir('CSCE585ProjectROI')


Here I'm uploading my dependencies

In [ ]:
!pip install reactpy reactpy-router flask llama-cpp-python TensorFlow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 21.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422320 sha256=a2b090003da1e2f0349286e835c0c3f0c1d59449d132b45721eff879683bfbb3
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [ ]:
import sys
print(sys.version)

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


Verified that the notebook is running from python 3.12.12

Uploading Specific libaries

In [ ]:
# Body of neccessary imports
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np #<-- May be optional not sure as of 10/13/25.
# import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

Uploaded the model from local computer

In [ ]:
from google.colab import files
model = files.upload()

Saving set1.py to set1.py


In [ ]:
from google.colab import files
apple = files.upload()

Saving apple_Metrics.csv to apple_Metrics.csv


In [ ]:
from google.colab import files
google = files.upload()

Saving google_Metrics.csv to google_Metrics.csv


Integtergate investing strategy into the data first Apple

Step 1: Split data into training group, validation and test sets

X : Features

y : Target variable of dataset

In [ ]:
import numpy as np
import pandas as pd
import io
from sklearn.model_selection import train_test_split

In [ ]:
for fn in apple.keys():
    apple = pd.read_csv(io.StringIO(apple[fn].decode('utf-8')))

In [ ]:
for fn in google.keys():
  google = pd.read_csv(io.StringIO(google[fn].decode('utf-8')))

Here I'm defining X and y values for the Apple dataset where my metrics will be X values and dates will be y value

I extracted my feature columns

In [ ]:
# Define the feature columns based on your intended inputs
feature_columns = [
    "Operating Cash Flow",
    "Capital Expenditure",
    "Free Cash Flow",
    "Total Debt",
    "Common Stockholder Equity",
    "Total Liabilities Net Minority Interest",
    "Total Assets",
    "Shares Outstanding",
    "Net Income Common Stockholders",
    "Diluted Average Shares",
    "Diluted EPS"
]

# --- Start of fix ---
# The 'apple' DataFrame needs to be transformed to have metrics as columns and dates as rows.
# Then, it needs to be merged with price data to get the 'Price' column.

# Transpose the 'apple' DataFrame: set 'Metric' as index, then transpose
apple_transformed_metrics = apple.set_index('Metric').T
apple_transformed_metrics.index.name = 'Date'

# Convert the index to datetime objects and sort (this will be tz-naive by default)
apple_transformed_metrics.index = pd.to_datetime(apple_transformed_metrics.index)
apple_transformed_metrics = apple_transformed_metrics.sort_index()

# Convert historical_data index to datetime and sort (if not already done)
# historical_data.index is already a DatetimeIndex but it is timezone-aware.
# We need to make it timezone-naive to match apple_transformed_metrics.index.
historical_data.index = historical_data.index.tz_localize(None) # Remove timezone information
historical_data = historical_data.sort_index()

# Extract 'Close' price as 'Price' from historical_data
price_data = historical_data['Close'].rename('Price')

# Convert price_data Series to a DataFrame so it merges as a column reliably
price_df = price_data.to_frame()

# Merge the transformed metrics with the price data.
# Use 'left' merge to keep all dates from apple_transformed_metrics, ensuring 'Price' column is created.
apple_transformed = apple_transformed_metrics.merge(price_df, left_index=True, right_index=True, how='left')

# Drop rows where 'Price' is NaN (i.e., no matching price data for that date)
apple_transformed = apple_transformed.dropna(subset=['Price'])

# Check if the resulting DataFrame is empty after dropping NaNs
if apple_transformed.empty:
    print("Warning: No common dates found between financial metrics and historical prices after merge and NaN removal.")
    print("X_data and y_data will be empty.")
    X_data = pd.DataFrame() # Return empty DataFrame
    y_data = pd.Series(dtype='float64') # Return empty Series
else:
    # Ensure all required feature columns exist and filter to only include those present
    existing_feature_columns = [col for col in feature_columns if col in apple_transformed.columns]

    # Extract X (features) and y (target 'Price')
    X_data = apple_transformed[existing_feature_columns]
    y_data = apple_transformed['Price']

# --- End of fix ---

print("Features (X_data) head:\n", X_data.head())
print("\nTarget (y_data) head:\n", y_data.head())

KeyError: ['Price']

In [ ]:
from sklearn.model_selection import train_test_split

# Perform the train-test split on the actual data
X_train_data, X_test_data, y_train_data, y_test_data = train_test_split(X_data, y_data, test_size=0.25, random_state=42)

# Display the shapes to confirm
print("Shape of X_train_data:", X_train_data.shape)
print("Shape of y_train_data:", y_train_data.shape)
print("Shape of X_test_data:", X_test_data.shape)
print("Shape of y_test_data:", y_test_data.shape)

NameError: name 'X_train_data' is not defined

Now I have to define my validation data which is 50% of combined dataset

In [ ]:
X_main = X_train_data, X_test_data
y_main = y_train_data, y_test_data

X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.25, random_state=42)

print(X_train_data, X_val, y_train, y_val)

NameError: name 'X_train_data' is not defined

In [ ]:
# Validation set is 50% of combined dataset
x_val, x_test, y_val, y_test = train_test_split(X_main,
                                                y_main,
                                                test_size=0.5,
                                                random_state=42)
print(x_val, x_test, y_val, y_test)

Apple Data

Training : X_train, y_train

Validation : X_main, y_main

Test : X_test, y_test

Step 2: Use data leakage prevention techniques such as walk forward cross validation

Update I'm going to go with standard walk forward validation since the current dataset we have is small

In [ ]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def perform_walk_forward_validation(apple, training_window_size, testing_window_size, step_size):
    errors = []
    predictions = []

    # Ensure data is a pandas Series or DataFrame with a time-based index
    # For this example, assuming 'data' is a Series where the index represents time

    for i in range(training_window_size, len(data) - testing_window_size + 1, step_size):
        # Define training and testing windows
        train_data = data.iloc[i - training_window_size : i]
        test_data = data.iloc[i : i + testing_window_size]

        # In a real scenario, you'd extract features (X) and target (y)
        # For this simple example, we'll just predict the next value based on previous values
        X_train = pd.DataFrame({'lag1': train_data.shift(1).dropna()})
        y_train = train_data.iloc[1:] # Target is the actual value at the next step

        X_test = pd.DataFrame({'lag1': test_data.shift(1).dropna()})
        y_test = test_data.iloc[1:]

        # Handle cases where X_train or X_test might be empty or too small
        if X_train.empty or X_test.empty:
            continue

        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_test)

        # Evaluate and store results
        mse = mean_squared_error(y_test, y_pred)
        errors.append(mse)
        predictions.extend(y_pred)

    return errors, predictions


training_window = 30 # Use 30 days for training
testing_window = 7   # Predict for the next 7 days
step = 7             # Move forward by 7 days in each iteration

errors, predictions = perform_walk_forward_validation(data, training_window, testing_window, step)

print(f"Mean Squared Errors for each fold: {errors}")
print(f"Average MSE: {sum(errors) / len(errors) if errors else 'N/A'}")

Step 3: Hypertune the parameters using the validation data set

I'm going to use the random search technique since we are working with financial stock data

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# Create a pipeline with scaling and the linear model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_model', Ridge()) # Or Lasso, etc.
])


In [ ]:
from scipy.stats import uniform, loguniform
from sklearn.linear_model import Ridge

# Define the search space
param_distributions = {
    'linear_model__alpha': loguniform(1e-4, 1e3), # Example for Ridge
    # Add other parameters for other models as needed
}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Initialize the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=100, # Number of random combinations to try
    cv=3, # Number of cross-validation folds, changed from 5 to 3
    scoring='neg_mean_squared_error', # Or other scoring metrics
    random_state=42,
    n_jobs=-1 # Use all available CPU cores
)

# Fit the search object to your training data using the correct variables
random_search.fit(X_train_data, y_train_data)


Since we are measuring numerical values it would be best to use regression based evaluations such as MSE

Fetch yfinance data then pass it thru the fucntion fetch market data so it can make future predicitions

In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
from datetime import timedelta

In [ ]:
from google.colab import files
investing_strategies = files.upload()

Lets apply the investing strategies to the trained model

In [ ]:
import pandas as pd
import io

In [ ]:
for fn in investing_strategies.keys():
    investing_strategies = pd.read_csv(io.StringIO(investing_strategies[fn].decode('utf-8')))

In [ ]:
import yfinance as yf

# Define the ticker symbol
ticker_symbol = "AAPL"

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Fetch historical market data
historical_data = ticker.history(period="1y")  # data for the last year
print("Historical Data:")
print(historical_data)

# Fetch basic financials
financials = ticker.financials.dropna()
print("\nFinancials:")
print(financials)

# Fetch stock actions like dividends and splits
actions = ticker.actions.dropna()
print("\nStock Actions:")
print(actions)

In [ ]:
import json

# Access the content of the 'initialExperiments.ipynb' file
notebook_bytes = investing_strategies['initialExperiments.ipynb']

# Decode the byte content to a UTF-8 string
notebook_string = notebook_bytes.decode('utf-8')

# Parse the decoded string as a JSON object
notebook_content = json.loads(notebook_string)

print("Notebook content loaded and parsed successfully.")
# You can inspect the keys of the parsed notebook content if needed
print(f"Keys in parsed notebook content: {notebook_content.keys()}")

In [ ]:
def strip_comments_from_line(line):
    """
    Removes Python comments from a single line of code.
    Handles full-line comments, inline comments, and avoids removing '#' within string literals.
    """
    # Handle full-line comments
    if line.strip().startswith('#'):
        return ""

    cleaned_line = []
    in_single_quote_string = False
    in_double_quote_string = False
    escape_char = False

    for i, char in enumerate(line):
        if char == '\\' and not escape_char:
            escape_char = True
            cleaned_line.append(char)
            continue

        if escape_char: # if previous char was an escape char, reset and append char
            escape_char = False
            cleaned_line.append(char)
            continue

        if char == "'" and not in_double_quote_string and not escape_char:
            in_single_quote_string = not in_single_quote_string
            cleaned_line.append(char)
        elif char == '"' and not in_single_quote_string and not escape_char:
            in_double_quote_string = not in_double_quote_string
            cleaned_line.append(char)
        elif char == '#' and not in_single_quote_string and not in_double_quote_string:
            # Comment found outside of a string literal
            return "".join(cleaned_line).rstrip()
        else:
            cleaned_line.append(char)

    return "".join(cleaned_line).rstrip()

print("Function `strip_comments_from_line` defined successfully.")

# Example Test Cases (optional, for verification)
# print(f"'# This is a comment' -> '{strip_comments_from_line('# This is a comment')}'")
# print(f"'print(\"Hello\") # inline comment' -> '{strip_comments_from_line('print(\"Hello\") # inline comment')}'")
# print(f"'text = \"#hashtag\"' -> '{strip_comments_from_line('text = \"#hashtag\"')}'")
# print(f"'variable = 123' -> '{strip_comments_from_line('variable = 123')}'")
# print(f"'print(\\'\#comment\\') # escaped hash' -> '{strip_comments_from_line('print(\\'\#comment\\') # escaped hash')}'")

Function `strip_comments_from_line` defined successfully.


In [ ]:
cleaned_cells = []

for cell in notebook_content['cells']:
    if cell['cell_type'] == 'code':
        cleaned_source = []
        for line in cell['source']:
            cleaned_line = strip_comments_from_line(line)
            if cleaned_line:
                cleaned_source.append(cleaned_line + '\n') # Add newline back if it was removed by rstrip and the line is not empty
        cell['source'] = cleaned_source
        cleaned_cells.append(cell)
    else:
        # For non-code cells (e.g., markdown), append them as is
        cleaned_cells.append(cell)

print("Notebook cells processed and comments removed from code cells.")
# The 'cleaned_cells' list now contains the processed notebook content.

Notebook cells processed and comments removed from code cells.


In [ ]:
cleaned_notebook_content = {}

# Copy all keys and values from the original notebook_content (excluding 'cells')
for key, value in notebook_content.items():
    if key != 'cells':
        cleaned_notebook_content[key] = value

# Assign the cleaned_cells list to the 'cells' key
cleaned_notebook_content['cells'] = cleaned_cells

print("New notebook JSON structure created successfully.")
# You can inspect the structure by printing cleaned_notebook_content.keys()
# print(cleaned_notebook_content.keys())


New notebook JSON structure created successfully.


In [ ]:
import json

# Print the JSON representation of the cleaned notebook content
print(json.dumps(cleaned_notebook_content, indent=2))

After I removed the comments from the inital experiments, I'm going to apply the investing strategies to the data

In [ ]:
invest_X = cleaned_notebook_content(X_main, X_test_data, X_train)

NameError: name 'X_main' is not defined

In [ ]:
invest_y = cleaned_notebook_content(y_main, y_test_data, y_train)

NameError: name 'y_main' is not defined